# Train set과 wiki corpus내 [UNK] 토큰의 발생 확인
- 대회 metric이 EM(Exact Match) 이기 때문에 답안 도출 과정 및 점수에 큰 영향을 미치는 [UNK] 토큰을 분석하고 가능한 최소화할 방법을 도모하였다

In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer
import pandas as pd

train_data = load_from_disk('train_dataset')['train']
valid_data = load_from_disk('train_dataset')['validation']
test_data = load_from_disk('test_dataset')

print(train_data)
print(valid_data)
print(test_data)

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 240
})
DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})


## train dataset 전처리

In [2]:
df = pd.DataFrame()
title, context, question, id, answers, document_id = train_data['title'],train_data['context'],train_data['question'], train_data['id'],train_data['answers'],train_data['document_id']
df.insert(0,'title',title)
df.insert(1,'context',context)
df.insert(2, 'question', question)
df.insert(3, 'id', id)
# df.insert(4, 'answer_start', answers['answer_start'][0])
# df.insert(5, 'answer_text', answers['text'][0])
df.insert(4, 'document_id', document_id)

df

,title,context,question,id,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,47334
...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,37764
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924


토크나이징 된 context의 예시를 일부 출력해 보았다.

In [3]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
for i in range(3):
    print(df['context'][i], end = '\n\n')
    print(tokenizer.tokenize(df['context'][i]))

미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05

['미국', '상의', '##원', '또는', '미국', '상원', '(', 'United', 'State', '##s', 'Se', '##n', '##ate', ')', '은', '양', '##원', '##제', '##인', '미국', '의회

train_dataset 내 context의 대부분이 [UNK] 를 포함하고 있음을 알 수 있다

In [4]:
contexts = df['context'].tolist()
#print(answers)
context_with_unk= 0
ttl_unks = 0
for context in contexts:
    temp = tokenizer.tokenize(context)
    if '[UNK]' in temp: context_with_unk += 1
    for token in temp:
        if token == '[UNK]': ttl_unks += 1
print('total number of contexts', len(df))
print('contexts including [UNK] :', context_with_unk)
print('contexts including [UNK] :', ttl_unks)

Token indices sequence length is longer than the specified maximum sequence length for this model (1020 > 512). Running this sequence through the model will result in indexing errors


total number of contexts 3952
contexts including [UNK] : 3732
contexts including [UNK] : 28279


### `\n` 제거
3952개의 dataset중 [UNK] 가 1개라도 포함된 context는 3732개로, 거의 대부분임을 확인하였다.  
일부 샘플을 눈으로 확인했을 때 \n\n과 이 [UNK] 를 구성함을 확인하고, 처리할 필요가 있음을 확인했다.  

In [5]:
context_n_rm = []
for context in contexts: #len(contexts)
    temp = context
    temp_n = temp.replace('\\n', ' ')
    temp_nn = temp_n.replace('\n', ' ')
#    print(temp_n)
    context_n_rm.append(temp_n)
print(len(context_n_rm))

3952


In [6]:
context_w_unk = 0
ttl_unks_del = 0
for context in context_n_rm:
    temp = tokenizer.tokenize(context)
    if '[UNK]' in temp: context_w_unk += 1
    for token in temp:
        if token == '[UNK]': ttl_unks_del += 1
print('contexts including [UNK] :', context_w_unk)
print('contexts including [UNK] :', ttl_unks_del)

contexts including [UNK] : 1546
contexts including [UNK] : 5471


28279개의 [UNK] 중 5471개를 제거할 수 있으며, 내용에도 영향을 미치지 않으니 `\n` 은 제거할 필요가 있다고 판단하였다.

### 한자 제거
많은 UNK토큰이 한자로부터 발생하지만 answer에 한자가 많이 포함되어 있으면 섣불리 제거할 수 없기에 우선 answer의 한자 분포를 확인해 보았다.  

In [9]:
def cut_punct(inp):
    print(inp)
    return inp[1:-1]

ans = train_data['answers']
ans_df = pd.DataFrame(ans)

ans_df['text'] = ans_df['text'].apply(lambda x: x[0])
ans_df['answer_start'] = ans_df['answer_start'].apply(lambda x: x[0])
df = pd.concat([df,ans_df], axis = 1)
df

,title,context,question,id,document_id,answer_start,text
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,18293,235,하원
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,51638,212,《경영의 실제》
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,5028,510,백성
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,34146,625,중국
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,47334,30,4개
...,...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275,317,지질 구조
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,37764,616,뉴저지
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730,255,칼 에케르트
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924,476,슬라브 신화


답변에 영향을 미칠 수 있기 때문에 valid set 의 결과도 마찬가지로 확인해 보았다

In [33]:
valid_data = load_from_disk('train_dataset')['validation']

#title, context, question, id, answers, document_id = train_data['title'],train_data['context'],train_data['question'], train_data['id'],train_data['answers'],train_data['document_id']

df_v = pd.DataFrame()
title, context, question, id, answers, document_id = valid_data['title'], valid_data['context'],valid_data['question'], valid_data['id'], valid_data['answers'], valid_data['document_id']
df_v.insert(0, 'title',title)
df_v.insert(1,'context',context)
df_v.insert(2, 'question', question)
df_v.insert(3, 'id', id)
df_v.insert(4, 'document_id', document_id)

ans_df_v = pd.DataFrame(answers)
ans_df_v['text'] = ans_df_v['text'].apply(lambda x: x[0])
ans_df_v['answer_start'] = ans_df_v['answer_start'].apply(lambda x: x[0])

df_v = pd.concat([df_v,ans_df_v], axis = 1)
df_v

,title,context,question,id,document_id,answer_start,text
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,9027,284,한보철강
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,51765,146,1871년
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,15694,517,나뭇잎
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,59536,1109,금대야
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,32991,386,수평적 관계
...,...,...,...,...,...,...,...
235,전단 (제나라),"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,53265,1084,제 양왕
236,티칼,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,12190,343,'일곱 개의 신전 광장'
237,윤치호,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,5326,247,미나미 지로
238,신잉카국,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,48575,563,"200,000명"


In [38]:
import re

answers = df['text'].tolist()
chi_include_answers = 0
ttl_chi_chars = 0
for answer in answers:
    temp = []
    for ch in re.findall('[\u4e00-\u9fff]+',answer):
        if ch != '':
            chi_include_answers += 1
            ttl_chi_chars += len(ch)

print('-----in train -----')
print('chinese included answers :', chi_include_answers)
print('total chinese characters :', ttl_chi_chars)

answers_v = df_v['text'].tolist()
chi_include_answers_v = 0
ttl_chi_chars_v = 0
for answer in answers_v:
    temp = []
    for ch in re.findall('[\u4e00-\u9fff]+',answer):
        if ch != '':
            chi_include_answers_v += 1
            ttl_chi_chars_v += len(ch)

print('-----in valid -----')
print('chinese included answers :', chi_include_answers_v)
print('total chinese characters :', ttl_chi_chars_v)


-----in train -----
chinese included answers : 49
total chinese characters : 120
-----in valid -----
chinese included answers : 6
total chinese characters : 18


**결론은 한자를 함부로 필터링하면 안되겠다는 것이었다.**   
train set에서는 3952개의 답변 중 49개로 1.24%가 한자를 포함했고  
valid set에서는 240개의 답변 중 6개로 2.5%가 한자를 포함했다.(valid set에서의 비율이 더 높다)  
따라서 유의미한 비율이라고 보고 한자 삭제는 진행하지 않기로 했다.  
대신 valid set은 240문항밖에 되지 않으니 간단히 답안들을 읽어보자

In [40]:
print(df_v['text'].tolist())

['한보철강', '1871년', '나뭇잎', '금대야', '수평적 관계', '옥음방송', '코칭 스티치', '복잡한 감염병', '스페인', '20세기 초', '"5월의 왕"', "'일급 비밀 프로젝트 2501'", '테헤란', '역사교육과정개발추진위원회', '1967년 11월 15일', '1965년', '아리크 부케', '〈중앙일보〉', '미타케성', '전체 4순위', '뇌물', '보통 유형 준융합성 천연두', '데스탱 장군', '박트리아', "'진전(陳田)'이라 새겨진 기와조각", '《국가》', '2011년 3월 19일', '유럽 포르투갈어', '메이저 릿지', '요희', '포드 극장', '만주국 관리', '『협동조합에 관하여』', '도고쿠', '몽키 D. 가프', '조류', '광주교도소', '작가 베게티우스', '가오슝 시', '쇠망치', '마리즈 교수', '런던', '이틀', '의상대사', '홋카이도', '‘우유의 바다’', '장어', '소련', '전치', '흑색육(黑色肉)', '욱', '독일인민당', '하기노 역', '회칠', '황강다리', '직업 교육', '공산당', '매년 음력 정월', '조계종', '베이징', '독일군', '1932년', '제서지전(齊西之戰)이후', '가루아', '출생 천궁도', '나말여초', '1939년', '창 절제술', '도버 밀', '여정현(呂正鉉)', '광배', '울산', '땅(은색)과 하늘(금색)을 맺고 푸는 권한', '저수지', '퀘이커 교', '피에르 오주로', '지분', '독일', '제 3자', '페이팔', '가리타 히사노리', '점수', '코스모케라톱스', "'사채회사'", '스위스', '"공유지의 비극"', '그 자체가 계급 모순을 가지는 체제', '해리 트루먼 대통령', '무위태수', '크라운라이터', '1990년 9월', '"인간과 요괴의 완전한 평등"', '경순왕', '도널드 트럼프 미국 대통령', '바르톨로메오 브뤼기에르', '불교', '경위대식 망원경', '남큐슈 자동차 전용 도로', '세실리아 페인',

In [ ]:
뭐 어쨌든 한자랑 \n 제거하고 나머지 unk는 얼마나 되나 확인해보자.

In [41]:
context_n_chi_rm = []
unk_list = []

for i in range(len(context_n_rm)):
    temp = re.sub('[\u4e00-\u9fff]+', '', context_n_rm[i])
    context_n_chi_rm.append(temp)
    tokens = tokenizer.tokenize(temp)
    if '[UNK]' in tokens:
        unk_list.append(i)

print(len(context_n_chi_rm))
print('context including UNKS', len(unk_list))

3952
context including UNKS 890


3952개 중 890개의 context만 UNK가 남았다. 확인해본 결과 대부분 고유명사(지명/인명), 영어의 한국어 표기(쉘터 등) 에서 발생하는 것으로 확인하였다. domain-specific dataset이 아니므로 경향성은 딱히 없다고 판단된다. 예시는 아래에서 확인할 수 있다.

In [159]:
for i in range(2488,2493):
    print(context_n_chi_rm[i])
    print(tokenizer.tokenize(context_n_chi_rm[i]))

역의 동쪽으로는 남북으로 뻗어 있는 언덕이 있고 인가는 없다. 남쪽에는 성토 위에 콘크리트로 덮인 유치선이 있고 그 앞에는 조잔케이 철도선의 폐선 자취가 도로변의 길고 숲을 이루면서 뻗어 있다.  역 서쪽은 주택지에서 지하철의 쉘터에 붙도록 단지군(올림픽 단지)이 형성되고 있다. 주변의 마코마나이 강 우안의 평지 외, 좌안의 언덕 위까지도 주택지가 조성되어 있다. 마코마나이 공원은 1972년에 삿포로 올림픽의 메인 회장이 되었다. 상업 시설 규제가 어려운 지역이므로 역 주변에 뚜렷한 상업 시설은 적어서 큰 상업 시설은 이 곳에서 버스로 10분 정도의 거리에 올림픽 대교 주변에 집중되어 있다.  * 국도 제453호선 * 뮤 크리스털 - 마코마나이 역에서 도보 4분. 마코마나이 지구의 유일한 대형 쇼핑몰 상업 시설. * 삿포로 시 미나미 구청 * 미나미 경찰서 마코마나이 단지 파출소 * 삿포로미나미 우체국 * 호쿠요 은행 마코마나이추오 지점 * 홋카이도 은행 마코마나이 지점 * 마코마나이 공원 (역에서 약 1km) * 에드윈 던 기념관
['역', '##의', '동쪽', '##으로', '##는', '남북', '##으로', '뻗', '##어', '있', '##는', '언덕', '##이', '있', '##고', '인가', '##는', '없', '##다', '.', '남쪽', '##에', '##는', '성토', '위', '##에', '콘크리트', '##로', '덮인', '유치', '##선', '##이', '있', '##고', '그', '앞', '##에', '##는', '조', '##잔', '##케이', '철도', '##선', '##의', '폐', '##선', '자취', '##가', '도로변', '##의', '길', '##고', '숲', '##을', '이루', '##면서', '뻗', '##어', '있', '##다', '.', '역', '서쪽', '##은', '주택', '##지', '##에서', '지하철', '##의', '[UNK]', '붙', '##도록', '단지', '#

따라서 \n 및 기타 특수문자는 처리하고, 한자를 처리하지 않은 dataset을 제작하는 것으로 결론지었다.